# GeekBrains, Recommendation Systems
# Lesson 2 Homework

**Импорт библилотек**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

**Датасет**

In [3]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [57]:
result = pd.read_pickle('predictions_basic.pkl')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[17214934, 12352370, 12158320, 846735, 1121862]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 6666, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 6666, 1098066]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[857381, 10461735, 904375, 1120777, 392314]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 6666, 826249]","[1082185, 981760, 1098066, 826249, 6666]"


In [59]:
for col in result.columns[1:]:
    print(type(result[col].iloc[0]))

<class 'numpy.ndarray'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


**Делим на трейн и тест**

In [4]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

### Решение Задания 1

In [30]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    items = np.array(items_weights['item_id'])
    weights = np.array(items_weights['weight'])
    # Используем параметр p
    recs = np.random.choice(items,
                            size=n,
                            replace=False,
                            p=weights)
    
    return recs.tolist()

In [27]:
%%time

# your_code
# Считаем продажи каждого товара, берем логарифм
# Полученные значения делаем положительными и приводим к единичной сумме
item_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()
item_weights['sales_value'] = np.log2(item_weights['sales_value'] + 0.001)
item_weights['sales_value'] = item_weights['sales_value'] - item_weights['sales_value'].min()
item_weights['sales_value'] = item_weights['sales_value'] / item_weights['sales_value'].sum()
item_weights = item_weights.rename(columns={'sales_value': 'weight'})
item_weights

Wall time: 272 ms


,item_id,weight
0,25671,0.000012
1,26081,0.000008
2,26093,0.000009
3,26190,0.000009
4,26355,0.000009
...,...,...
86860,17381856,0.000000
86861,17382205,0.000011
86862,17383227,0.000010
86863,17827644,0.000010


In [79]:
%%time

result['w_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(item_weights, n=5))
result.head(2)

Wall time: 4.13 s


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,w_random_recommendation,random_recommendation_5000,popular_recommendation_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[17214934, 12352370, 12158320, 846735, 1121862]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 6666, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 6666, 1098066]","[5572704, 15596961, 1091616, 97206, 9485347]","[8177622, 919644, 1040197, 950935, 1057365]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[857381, 10461735, 904375, 1120777, 392314]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 6666, 826249]","[1082185, 981760, 1098066, 826249, 6666]","[1478506, 13157977, 930606, 7410035, 9368118]","[1131400, 1005493, 1071277, 5570048, 934369]","[6534178, 6533889, 1029743, 6534166, 1082185]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [60]:
from metrics import precision_at_k

In [68]:
%%time

# your_code
rec_metrics = {}
for col in result.columns[1:]:
    rec_metrics[col] = round(result.apply(lambda row: precision_at_k(row[col], row['actual']), axis=1).mean(), 4)

Wall time: 1.89 s


In [69]:
pd.DataFrame(pd.Series(rec_metrics)).T

,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,w_random_recommendation
0,1.0,0.0006,0.1552,0.1368,0.1329,0.139,0.0012


Weighted Random Recommendation показал себя в 2 раза лучше, чем Random Recommendation, но все равно плохо по сравнению с остальными.

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

**Бейзлайны**

**1. Random recommendation**

In [71]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [72]:
%%time

items = data_train['item_id'].iloc[:5000].unique()

result['random_recommendation_5000'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

result.head(2)

Wall time: 589 ms


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,w_random_recommendation,random_recommendation_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[17214934, 12352370, 12158320, 846735, 1121862]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 6666, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 6666, 1098066]","[828732, 6555617, 958218, 13417835, 15830893]","[8177622, 919644, 1040197, 950935, 1057365]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[857381, 10461735, 904375, 1120777, 392314]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 6666, 826249]","[1082185, 981760, 1098066, 826249, 6666]","[12695533, 954759, 1202678, 72163, 822325]","[1131400, 1005493, 1071277, 5570048, 934369]"


**2. Popularity-based recommendation**

In [73]:
def popularity_recommendation_5000(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular = popular.sort_values('sales_value', ascending=False).iloc[:5000]
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [75]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation_5000(data_train, n=5)

result['popular_recommendation_5000'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

Wall time: 300 ms


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,w_random_recommendation,random_recommendation_5000,popular_recommendation_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[17214934, 12352370, 12158320, 846735, 1121862]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 6666, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 6666, 1098066]","[828732, 6555617, 958218, 13417835, 15830893]","[8177622, 919644, 1040197, 950935, 1057365]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[857381, 10461735, 904375, 1120777, 392314]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 6666, 826249]","[1082185, 981760, 1098066, 826249, 6666]","[12695533, 954759, 1202678, 72163, 822325]","[1131400, 1005493, 1071277, 5570048, 934369]","[6534178, 6533889, 1029743, 6534166, 1082185]"


**3. Weighted Random Recommendation**

In [77]:
%%time

# your_code
# Считаем продажи каждого товара, берем логарифм
# Полученные значения делаем положительными и приводим к единичной сумме
item_weights_5000 = data_train.groupby('item_id')['sales_value'].sum().reset_index()
item_weights_5000 = item_weights_5000.sort_values('sales_value', ascending=False).iloc[:5000]
item_weights_5000['sales_value'] = np.log2(item_weights_5000['sales_value'] + 0.001)
item_weights_5000['sales_value'] = item_weights_5000['sales_value'] - item_weights_5000['sales_value'].min()
item_weights_5000['sales_value'] = item_weights_5000['sales_value'] / item_weights_5000['sales_value'].sum()
item_weights_5000 = item_weights_5000.rename(columns={'sales_value': 'weight'})
item_weights_5000

Wall time: 353 ms


,item_id,weight
55470,6534178,2.034427e-03
55430,6533889,1.376497e-03
28895,1029743,1.342572e-03
55465,6534166,1.296012e-03
34707,1082185,1.255600e-03
...,...,...
72352,12188446,1.651475e-07
37958,1112387,1.341899e-07
23584,981716,4.129621e-08
51978,5575861,1.032464e-08


In [80]:
%%time

result['w_random_recommendation_5000'] = result['user_id'].apply(lambda x: weighted_random_recommendation(item_weights_5000, n=5))
result.head(2)

Wall time: 853 ms


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,w_random_recommendation,random_recommendation_5000,popular_recommendation_5000,w_random_recommendation_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[17214934, 12352370, 12158320, 846735, 1121862]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 6666, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 6666, 1098066]","[5572704, 15596961, 1091616, 97206, 9485347]","[8177622, 919644, 1040197, 950935, 1057365]","[6534178, 6533889, 1029743, 6534166, 1082185]","[845208, 5568197, 857503, 922127, 851066]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[857381, 10461735, 904375, 1120777, 392314]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 6666, 826249]","[1082185, 981760, 1098066, 826249, 6666]","[1478506, 13157977, 930606, 7410035, 9368118]","[1131400, 1005493, 1071277, 5570048, 934369]","[6534178, 6533889, 1029743, 6534166, 1082185]","[851146, 1041259, 1139525, 847374, 8116306]"


In [82]:
%%time

# your_code
rec_metrics_5000 = {}
for col in result.columns[1:]:
    rec_metrics_5000[col] = round(result.apply(lambda row: precision_at_k(row[col], row['actual']), axis=1).mean(), 4)

Wall time: 2.75 s


In [83]:
pd.DataFrame(pd.Series(rec_metrics_5000)).T

,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,w_random_recommendation,random_recommendation_5000,popular_recommendation_5000,w_random_recommendation_5000
0,1.0,0.0006,0.1552,0.1368,0.1329,0.139,0.0012,0.0046,0.1552,0.0125


Популярные рекомендации не стали лучше, что вполне понятно. Случайные невзвешенные и взвешенные рекомендации улучшились значительно (что понятно для взвешенных рекомендаций).

**Улучшенные ItemItemRecommender**

In [ ]:
# your_code